### Import required libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('data/life_tables.xlsx')
df.shape

(2274, 6)

In [3]:
df.head()

,table,gender,age,qx,lx,dx
0,AT2000 (Suavizada 10%),F,0,0.001615,1000000.000000,1615.000000
1,AT2000 (Suavizada 10%),F,1,0.000680,998385.000000,678.901800
2,AT2000 (Suavizada 10%),F,2,0.000353,997706.098200,352.190253
3,AT2000 (Suavizada 10%),F,3,0.000261,997353.907947,260.309370
4,AT2000 (Suavizada 10%),F,4,0.000209,997093.598577,208.392562


In [4]:
df.columns

Index(['table', 'gender', 'age', 'qx', 'lx', 'dx'], dtype='object')

In [5]:
tables = list(df['table'].unique())

[{'value':i+1, 'label':tb} for i, tb in enumerate(tables)]

[{'label': ' AT2000 (Suavizada 10%)', 'value': 1},
 {'label': ' AT2000', 'value': 2},
 {'label': 'AT-49', 'value': 3},
 {'label': 'BR-EMSmt-v.2010', 'value': 4},
 {'label': 'BR-EMSsb-v.2010', 'value': 5},
 {'label': 'BR-EMSmt-v.2015', 'value': 6},
 {'label': 'BR-EMSsb-v.2015', 'value': 7},
 {'label': 'IBGE 2006', 'value': 8},
 {'label': 'IBGE 2008', 'value': 9},
 {'label': 'IBGE 2007', 'value': 10},
 {'label': 'IBGE 2009', 'value': 11}]

In [6]:
df_interest = pd.read_excel("data/risk_free.xlsx")

In [7]:
df_interest[df_interest['month'] == df_interest['month'].max()]['selic_year'].values[0]/100

0.0416

In [8]:
df['lx'].values.min()

0.0

In [9]:
df_ = df.query('table == " AT2000" and gender == "M"').reset_index(drop=True).copy()

In [10]:
df_['lx'].values.shape

(116,)

In [11]:
def pv_calc(i_rate, n):
    '''
    This function calculates the present value factor v
    Input:
        i_rate: Interest rate --> float
        n: period --> int or np.array
    Output:
        A float or a vector of floats with the factor (1+rate)**(-n)
    '''
    return 1/(1+i_rate)**n

def calc_Dx(age, lx, i_rate):
    '''
    This function calculated the Dx commutation
    Input:
          age: a value or a vector of age values --> int or np.array
          lx: a value or a vector with the number of survivals in an specific age --> int or np.array
          i_rate: Interest rate --> float
    Output:
          An np.array with Dx commutation
    '''
    return lx*pv_calc(i_rate, age)

def calc_Cx(age, dx, i_rate):
    '''
    This function calculated the Cx commutation
    Input:
          age: a value or a vector of age values --> int or np.array
          dx: a value or a vector with the number of deaths in an specific age --> int or np.array
          i_rate: Interest rate --> float
    Output:
          An np.array with Cx commutation
    '''
    age_ = age + 1
    return dx*pv_calc(i_rate, age_)
    
def calc_Mx(Cx, ini=0):
    '''
    This function calculated the Nx commutation
    Input:
          Cx: An np.array with Dx commutations calculated --> np.array
          ini: Starting point of the commutation (it would be the first age to be used) --> int
    Output:
          An np.array with Mx commutation
    '''
    size = Cx.shape[0]
    if ini > size - 1:
        return 0
    else:
        Cx_ = Cx[ini:].copy()
        return Cx_[::-1].cumsum()[::-1]

def calc_Nx(Dx, ini=0):
    '''
    Thist function calculated the Nx commutation
    Input:
          Dx: An np.array with Dx commutations calculated --> np.array
          ini: Starting point of the commutation (it would be the first age to be used) --> int
    Output:
          An np.array with Nx commutation
    '''
    size = Dx.shape[0]
    if ini > size - 1:
        return 0
    else:
        Dx_ = Dx[ini:].copy()
        return Dx_[::-1].cumsum()[::-1]

def comut_calc(df, i_rate, which="Dx"):
    '''
    This function calculates Dx and Cx commutations based on a life table
    Input:
          df: life table dataframe --> pd.DaraFrame
          i_rate: interest rate --> float
          which: Commutation to be calculated (Dx or Cx) --> string
    Output:
          An array of commutations --> np.array
    '''
    age = df['age'].values
    lx = df['lx'].values
    dx = df['dx'].values
    
    if which == "Dx":
        return calc_Dx(age, lx, i_rate)
    elif which == "Cx":
        return calc_Cx(age, dx, i_rate)

In [12]:
Dx = comut_calc(df_, 0.0416)
Cx = comut_calc(df_, 0.0416, 'Cx')

Mx = calc_Mx(Cx)
Nx = calc_Nx(Dx)

In [13]:
Dx.min()

0.001662205643568587

In [14]:
def calc_pup(NMx, Dx, dif, term, antecip=True, age=None, prod='a'):
    '''
    This function calcultes the net sigle premium of an Annuity, Insurance or Endowment
    Input: 
           NMx: Commutation NMX (Nx for annuities, Mx for Insurance) --> np.array
           Dx: Dx Commutation  --> np.array
           dif: deferment period --> int
           term: product term --> int
           antecip: True if the annuity is anticipated --> Boolean
           age: List of ages to be selected, not required --> List
           prod: A, a, E for Insurance, annuity, Endowment
           
    Output: 
          A vector np.array of net sigle premiums --> np.array
    '''

    #Antecip or postecip, for insurance it is always post
    add_one = 0 if antecip == True or prod == 'A' else 1
    
    #Copes with the differences between whole life and term products
    if term == np.inf:
        ini_ = dif + add_one if dif + add_one > 0 else None
        ini_sign = ini_ if ini_ is None else - ini_
        #Insurance and annuities
        #(Mx+n) / Dx or (Nx+n) / Dx, n = 0,1,2,3,...
        result = NMx[ini_:] / Dx[:ini_sign] 
    else:
        #Copes with deferment period
    
        if prod == 'D' or prod == 'd':
            dif = 0
            add_one = 0
            
        #Avoid a dif + term greater than life table's size
        term_ = min(term, NMx.shape[0] - 1 - dif - add_one)
        ini_dif = dif + term_ + add_one
        ini_term = dif + add_one
        len_ = NMx[ini_dif:].shape[0]
        fin_term = ini_term + len_
        
        if prod == 'D':
            #Endowment
            #(Mx+m - Mx+m + Dx+m) / Dx
            result = (NMx[ini_term:fin_term] - NMx[ini_dif:] + Dx[ini_dif:]) / Dx[:len_]
        elif prod == 'd':
            #Endowment
            #(Dx+m) / Dx
            result = Dx[ini_dif:] / Dx[:len_]
        else:
            #Insurance and annuities
            #(Mx+n - Mx+n+m) / Dx or (Nx+n - Nx+n+m) / Dx, n = 0,1,2,3,...
            result = (NMx[ini_term:fin_term] - NMx[ini_dif:]) / Dx[:len_]
            
    return result if age is None else result[age]

In [20]:
dotal_misto = calc_pup(Mx, Dx, 0, 10, True, None, 'D')
dotal_puro  = calc_pup(Mx, Dx, 0, 20, True, None, 'd')
seguro = calc_pup(Mx, Dx, 2, 20, True, None, 'A')
anuidade_antecip = calc_pup(Nx, Dx, 2, 15, True, None, 'a')
anuidade_postecip = calc_pup(Nx, Dx, 2, 15, True, None, 'a')

In [30]:
max_shape = max(dotal_misto.shape[0], 
                dotal_puro.shape[0], 
                seguro.shape[0], 
                anuidade_antecip.shape[0], 
                anuidade_postecip.shape[0])

In [47]:
list_prods = [dotal_misto, dotal_puro, seguro, 
              anuidade_antecip, anuidade_postecip]

list_prods_out = []

for prod in list_prods:
    zeros = np.zeros((max_shape, ), dtype=prod.dtype)
    zeros[:prod.shape[0]] = prod
    
    list_prods_out.append(zeros)
    

In [49]:
len(list_prods_out)

5

In [51]:
out_simu = pd.DataFrame().from_dict({'dotal_misto_10':list_prods_out[0],
                          'dotal_puro_20':list_prods_out[1],
                          'seguro_2_20':list_prods_out[2],
                          'anu_antecip_2_15':list_prods_out[3],
                          'anu_postecip_2_15':list_prods_out[4]})

In [52]:
out_simu.to_excel('data/simu.xlsx', index=False)
